In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PyTorch Image Classification Multi-Node Distributed Data Parallel Training on CPU using Vertex Training with Custom Container

<table align="left">

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pytorch_image_classification_distributed_data_parallel_training_with_vertex_sdk/multi_node_ddp_gloo_vertex_training_with_custom_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>                                                                                               <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pytorch_image_classification_distributed_data_parallel_training_with_vertex_sdk/multi_node_ddp_gloo_vertex_training_with_custom_container.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview


This tutorial demonstrates how to create a distributed PyTorch training job using Vertex SDK and custom containers. This can help your training job scale to handle a large amount of data.


### Dataset

The dataset used for this tutorial is the <a href="https://en.wikipedia.org/wiki/MNIST_database">MNIST database</a>. It is a large database of handwritten digits that is commonly used for training various image processing systems.


### Objective

In this tutorial, you learn how to use create a distributed PyTorch training job using Vertex SDK and custom containers. You will set up GCP to use a custom container, a Vertex Tensorboard Instance and run a custom training job.

This tutorial uses the following Google Cloud ML services:

- `Vertex SDK`
- `Vertex Tensorboard`
- `CustomContainerTrainingJob`

The steps performed include:

- Setting up your GCP project : Setting up the PROJECT_ID, REGION & SERVICE_ACCOUNT
- Creating a cloud storage bucket
- Building Custom Container using Container Registry and Docker
- Create a Vertex Tensorboard Instance to store your Vertex AI experiment
- Run a Vertex SDK CustomContainerTrainingJob


## Costs
 
This tutorial uses billable components of Google Cloud:

Vertex AI
Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/),
        to generate a cost estimate based on your projected usage.


### Install additional packages

Install the latest version of Vertex SDK for Python.

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform

In [ ]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Service Account
If you don't know your service account, try to get your service account using gcloud command by executing the second cell below.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your GCP project id from gcloud
    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
    print("Service Account:", SERVICE_ACCOUNT)

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

In [ ]:
content_name = "pt-img-cls-multi-node-ddp-cust-cont"

## Local Training


In [ ]:
! ls trainer

In [ ]:
! cat trainer/requirements.txt

In [ ]:
! pip install -r trainer/requirements.txt

In [ ]:
! cat trainer/task.py

In [ ]:
%run trainer/task.py --epochs 5 --no-cuda --local-mode 

In [ ]:
! ls ./tmp

Clean up temporary files

In [ ]:
! rm -rf ./tmp

## Vertex Training using Vertex SDK and Custom Container

### Build Custom Container

In [ ]:
hostname = "gcr.io"
image_name = content_name
tag = "latest"

custom_container_image_uri = f"{hostname}/{PROJECT_ID}/{image_name}:{tag}"

In [ ]:
! cd trainer && docker build -t $custom_container_image_uri -f Dockerfile .

In [ ]:
! docker run --rm $custom_container_image_uri --epochs 5 --no-cuda --local-mode

In [ ]:
! docker push $custom_container_image_uri

In [ ]:
! gcloud container images list --repository $hostname/$PROJECT_ID

### Initialize Vertex SDK

In [ ]:
from google.cloud import aiplatform

aiplatform.init(
    project=PROJECT_ID,
    staging_bucket=BUCKET_URI,
    location=REGION,
)

### Create a Vertex Tensorboard Instance

In [ ]:
content_name = content_name + "-cpu"

In [ ]:
tensorboard = aiplatform.Tensorboard.create(
    display_name=content_name,
)

#### Option: Use a Previously Created Vertex Tensorboard Instance

```
tensorboard_name = "Your Tensorboard Resource Name or Tensorboard ID"
tensorboard = aiplatform.Tensorboard(tensorboard_name=tensorboard_name)
```

### Run a Vertex SDK CustomContainerTrainingJob

In [ ]:
display_name = content_name
gcs_output_uri_prefix = f"{BUCKET_URI}/{display_name}"

replica_count = 4
machine_type = "n1-standard-4"

args = [
    "--backend",
    "gloo",
    "--no-cuda",
    "--batch-size",
    "128",
    "--epochs",
    "25",
]

In [ ]:
custom_container_training_job = aiplatform.CustomContainerTrainingJob(
    display_name=display_name,
    container_uri=custom_container_image_uri,
)

In [ ]:
custom_container_training_job.run(
    args=args,
    base_output_dir=gcs_output_uri_prefix,
    replica_count=replica_count,
    machine_type=machine_type,
    tensorboard=tensorboard.resource_name,
    service_account=SERVICE_ACCOUNT,
)

In [ ]:
print(f"Custom Training Job Name: {custom_container_training_job.resource_name}")
print(f"GCS Output URI Prefix: {gcs_output_uri_prefix}")

### Training Output Artifact

In [ ]:
! gsutil ls $gcs_output_uri_prefix

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Artifacts
- Cloud Storage Bucket

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

! gsutil rm -rf $gcs_output_uri_prefix

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI